# Veritasium - Transcriptions

## Downloading & Importing Libraries

In [1]:
!pip install openai langchain langchain_community langchain_openai yt-dlp ffmpeg-python transformers torchaudio

In [2]:
import os
import gc
import requests
import openai
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

import json
from google.colab import files
from google.colab import userdata
from google.colab import runtime

import re
import string
import nltk

import torch
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import yt_dlp
import ffmpeg
import subprocess


In [4]:
OPENAI_API_KEY = userdata.get('Ironhack-GPT')
GCC_API_KEY = userdata.get('BH-GCC')
HF_TOKEN = userdata.get('HF')


os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['HF_TOKEN'] = HF_TOKEN

## Veritasium videos pre-processing

In [5]:
# Load videos metadata
with open('/content/0-videos_metadata.json', 'r') as file:
    videos = json.load(file)

# Load categorized videos metadata
with open('/content/1-categorized_videos_metadata.json', 'r') as file:
    categorized_videos = json.load(file)

### Getting Transcriptions

#### Test
''' uncomment for trying out'''

In [6]:
# # Function to download video from YouTube using yt-dlp
# def download_youtube_video(video_url, output_path="video.mp4"):
#     ydl_opts = {
#         'format': 'bestaudio/best',
#         'outtmpl': output_path,
#     }
#     with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#         ydl.download([video_url])
#     return output_path

# # Function to extract audio from video using ffmpeg
# def extract_audio_from_video(video_path, output_audio_path="audio.wav"):
#     input_stream = ffmpeg.input(video_path)
#     audio_stream = ffmpeg.output(input_stream, output_audio_path, format='wav', acodec='pcm_s16le', ac=1, ar='16k')
#     ffmpeg.run(audio_stream)
#     return output_audio_path

# # Example video
# video_url = "https://www.youtube.com/watch?v=SQggDnScsvI"
# video_path = download_youtube_video(video_url)
# audio_path = extract_audio_from_video(video_path)

# print(f"Video downloaded to: {video_path}")
# print(f"Audio extracted to: {audio_path}")


In [7]:
# # Check for GPU availability
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load the processor and model, and move the model to the GPU
# processor = WhisperProcessor.from_pretrained("openai/whisper-small")
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)

# # Function to load audio and resample it to 16kHz
# def load_audio(file_path):
#     audio, sample_rate = torchaudio.load(file_path)
#     audio = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(audio)
#     return audio.squeeze()

# # Function to chunk audio with overlap
# def chunk_audio_with_overlap(audio, chunk_size=16000*30, overlap=16000*5):  # 30-second chunks with 5-second overlap
#     chunks = []
#     start = 0
#     while start < len(audio):
#         end = start + chunk_size
#         chunks.append(audio[start:end])
#         start = end - overlap
#     return chunks

# # Function to transcribe audio
# def transcribe_audio_chunk(audio_chunk):
#     inputs = processor(audio_chunk, return_tensors="pt", sampling_rate=16000, language='en')
#     inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU
#     with torch.no_grad():
#         logits = model.generate(**inputs)
#     transcription = processor.batch_decode(logits, skip_special_tokens=True)[0]
#     return transcription

# # Load and chunk the audio with overlap
# audio = load_audio(audio_path)
# audio_chunks = chunk_audio_with_overlap(audio)

# # Transcribe each chunk and combine transcriptions
# full_transcription = ""
# for chunk in audio_chunks:
#     transcription = transcribe_audio_chunk(chunk)
#     full_transcription += transcription + " "

# # Example metadata with full transcription  // Testing a short video
# video_metadata = {
#     "videoId": "SQggDnScsvI", 
#     "title": "How Do Chameleons Change Color?",
#     "transcription": full_transcription.strip()
# }

# print(video_metadata)


In [8]:
'''

trash
trash
trash
trash

'''
gc.collect()

0

#### Full transcriptions


In [14]:
import os
import subprocess
import torch
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import yt_dlp
import openai

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the processor and model, and move the model to the GPU
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium").to(device)

# Function to download the video and manually extract audio using ffmpeg
def download_video(url, video_output="video.mp4", audio_output="audio.wav"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': video_output,
        'prefer_ffmpeg': True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.download([url])

    # Manually extract audio using ffmpeg
    if os.path.exists(video_output):
        ffmpeg_command = f"ffmpeg -i {video_output} -vn -acodec pcm_s16le -ar 16000 -ac 1 {audio_output}"
        try:
            subprocess.run(ffmpeg_command, shell=True, check=True)
            if os.path.exists(audio_output):
                return video_output, audio_output
            else:
                print(f"Failed to extract audio to {audio_output}")
                return video_output, None
        except subprocess.CalledProcessError as e:
            print(f"ffmpeg error: {e}")
            return video_output, None
    else:
        print(f"Failed to download video to {video_output}")
        return None, None

# Function to load audio and resample it to 16kHz
def load_audio(file_path):
    if os.path.exists(file_path):
        audio, sample_rate = torchaudio.load(file_path)
        audio = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(audio)
        return audio.squeeze()
    else:
        raise RuntimeError(f"Audio file {file_path} not found.")

# Function to chunk audio with no overlap
def chunk_audio_with_no_overlap(audio, chunk_size=16000*15):  # 15-second chunks with no overlap performed best!
    chunks = []
    start = 0
    chunk_index = 0
    while start < len(audio):
        end = start + chunk_size
        chunk = audio[start:end]
        chunks.append(chunk)
        print(f"Chunk {chunk_index}: Start={start}, End={end}")
        start = end
        chunk_index += 1
    return chunks

# Function to transcribe audio
def transcribe_audio_chunk(audio_chunk):
    inputs = processor(audio_chunk, return_tensors="pt", sampling_rate=16000, language='en')
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():
        logits = model.generate(**inputs)
    transcription = processor.batch_decode(logits, skip_special_tokens=True)[0]
    return transcription

# Main function to process each video
def process_video(video):
    url = video['url']
    try:
        video_file, audio_file = download_video(url, "video.mp4", "audio.wav")

        if audio_file is None:
            raise RuntimeError("Audio extraction failed.")

        # Load and chunk the audio
        audio = load_audio(audio_file)
        audio_chunks = chunk_audio_with_no_overlap(audio)

        full_transcription = ""
        for i, chunk in enumerate(audio_chunks):
            transcription = transcribe_audio_chunk(chunk)
            print(f"Chunk {i} transcription: {transcription}")
            full_transcription += transcription + "\n"  # Add a newline character after each chunk's transcription

        # Update video metadata with transcription
        video['transcription'] = full_transcription.strip()

        # Clean up downloaded files
        if os.path.exists(video_file):
            os.remove(video_file)
        if os.path.exists(audio_file):
            os.remove(audio_file)

    except Exception as e:
        print(f"Error processing video {video['videoId']}: {e}")

# Iterate and process each video
for category, videos in categorized_videos.items():
    for video in videos:
        process_video(video)


Output hidden; open in https://colab.research.google.com to view.

In [15]:
categorized_videos

Output hidden; open in https://colab.research.google.com to view.

In [17]:
# Function to save categorized videos metadata to a JSON file
def save_categorized_videos_metadata_transcribed(categorized_videos, filename="2-transcribed_videos_metadata.json"):
    with open(filename, 'w') as f:
        json.dump(categorized_videos, f, indent=4)

# Save the metadata to a file
save_categorized_videos_metadata_transcribed(categorized_videos)

# Download the JSON file
files.download('2-transcribed_videos_metadata.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>